In [4]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import datetime
import seaborn as sns
import pickle
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
%matplotlib inline
from sklearn.utils import resample
import re
import nltk
from sklearn.linear_model import LogisticRegressionCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import mean_squared_error, r2_score

In [5]:
!pip install sqlalchemy
!pip install pymssql

In [6]:
import sqlalchemy as sa
import pandas as pd
from sqlalchemy import create_engine


In [7]:
Username = 'admin'
Password = 'amazing_people'
Host = 'alphacode-explore.ccwgqdqrrmvt.eu-west-1.rds.amazonaws.com'
Port = '1433'
Database = 'Tebogo'

In [8]:
connection = create_engine(
    'mssql+pymssql://' +
    Username + ':' + Password + '@' + Host + ':' + Port + '/' + Database
)

In [9]:
sql_query1 = "USE Tebogo Select * from bank_statements"

In [10]:
train = pd.read_sql_query(sql_query1, connection)
train.head()

,Transaction_Date,Posting_Date,Description,Debits,Credits,Category,Column_7,CATEGORIES,Column_9
0,2020/02/10,2020/02/10,MTN DATA BUNDLES,499,None,COMMUNICATION,None,HOME,None
1,2020/02/08,2020/02/09,APPLE.COM/BILL CORK IE,74.99,None,ENTERTAINMENT & EAT OUT,None,FOOD,None
2,2020/02/07,2020/02/09,PARKHURST HARDWARE RE PARKHURST ZA,75,None,HOME,None,TRANSPORT,None
3,2020/02/07,2020/02/08,SEATTLE RENNIES HOUSE JOHANNESBURG ZA,31.5,None,ENTERTAINMENT & EAT OUT,None,LIFE AND HEALTH,None
4,2020/02/07,2020/02/08,FLM RENNIE HOUSE EATER BRAAMFONTEIN ZA,29.98,None,ENTERTAINMENT & EAT OUT,None,COMMUNICATION,None


In [11]:
train.tail()

,Transaction_Date,Posting_Date,Description,Debits,Credits,Category,Column_7,CATEGORIES,Column_9
938,None,2020-02-02,IBANK PAYMENT,200,None,GIFTS AND DONATIONS,None,None,None
939,None,2020-02-02,POS PURCHASE BP Filling Sta,81.6,None,TRANSPORT,None,None,None
940,None,2020-02-02,POS PURCHASE BP Filling Sta,28,None,TRANSPORT,None,None,None
941,None,2020-02-01,ACB DEBIT:EXTERNAL VIRGIN ACT,297.5,None,LIFE AND HEALTH,None,None,None
942,None,2020-02-01,MANAGEMENT FEE,29,None,FEES,None,None,None


In [12]:

y=train.Category.apply(lambda x: x.lower())
y.head()


0              communication
1    entertainment & eat out
2                       home
3    entertainment & eat out
4    entertainment & eat out
Name: Category, dtype: object

In [13]:
y.value_counts()

transport                  295
fees                       216
entertainment & eat out    174
communication               68
food                        42
home                        32
other                       28
short-term credit           27
life and health             19
saving and investment       17
gifts & donations           15
education                    7
interest                     1
gifts and donations          1
income                       1
Name: Category, dtype: int64

In [14]:
y_unique = y.value_counts()
y_unique = pd.DataFrame(y_unique)
y_unique.head()

,Category
transport,295
fees,216
entertainment & eat out,174
communication,68
food,42


In [15]:
y_unique = y_unique.reset_index(drop=False)
y_unique.head()

,index,Category
0,transport,295
1,fees,216
2,entertainment & eat out,174
3,communication,68
4,food,42


In [16]:
y_unique.drop(['Category'], axis=1, inplace=True)  
y_unique['New_ID'] = y_unique.index
y_unique.head()

,index,New_ID
0,transport,0
1,fees,1
2,entertainment & eat out,2
3,communication,3
4,food,4


In [17]:
len(y_unique)

15

In [18]:
y =  pd.DataFrame(y)
y.head()

,Category
0,communication
1,entertainment & eat out
2,home
3,entertainment & eat out
4,entertainment & eat out


In [19]:
y.Category.values

array(['communication', 'entertainment & eat out', 'home',
       'entertainment & eat out', 'entertainment & eat out',
       'entertainment & eat out', 'entertainment & eat out',
       'entertainment & eat out', 'entertainment & eat out',
       'entertainment & eat out', 'saving and investment',
       'entertainment & eat out', 'entertainment & eat out',
       'entertainment & eat out', 'saving and investment',
       'entertainment & eat out', 'entertainment & eat out',
       'entertainment & eat out', 'entertainment & eat out', 'other',
       'saving and investment', 'life and health', 'other',
       'entertainment & eat out', 'entertainment & eat out',
       'entertainment & eat out', 'entertainment & eat out',
       'life and health', 'saving and investment',
       'saving and investment', 'communication', 'life and health',
       'short-term credit', 'entertainment & eat out',
       'entertainment & eat out', 'entertainment & eat out',
       'entertainment & eat out

In [20]:
mapping = dict(y_unique[['index', 'New_ID']].values)
y['Item_ID'] = y.Category.map(mapping)
y.head()

,Category,Item_ID
0,communication,3
1,entertainment & eat out,2
2,home,5
3,entertainment & eat out,2
4,entertainment & eat out,2


In [21]:
y.drop(['Category'], axis=1, inplace=True)
y.head()

,Item_ID
0,3
1,2
2,5
3,2
4,2


In [23]:
len(y)

943

In [24]:
features = train.Description.values
features2 = train.Description.values

In [25]:

features[:5]

array(['MTN DATA BUNDLES', 'APPLE.COM/BILL CORK IE',
       'PARKHURST HARDWARE    RE PARKHURST ZA',
       'SEATTLE RENNIES HOUSE JOHANNESBURG ZA',
       'FLM RENNIE HOUSE EATER BRAAMFONTEIN ZA'], dtype=object)

In [26]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Oratile\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Oratile\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [27]:
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

lem = WordNetLemmatizer()
# ps = PorterStemmer()
corpus =[]
# cleaning text
for i in range(0,len(features)):

    item = re.sub('[^a-zA-Z]', ' ',str(features)) 
    item = item.lower()
    item = item.split()
    stop_wrds = set(stopwords.words('english'))
    item = [lem.lemmatize(word, "v") for word in item if not word in stop_wrds]
    item = ' '.join(item)
    corpus.append(item)

new_features = corpus

In [28]:
new_features[0]

'mtn data bundle apple com bill cork ie parkhurst hardware parkhurst za seattle rennies house johannesburg za flm rennie house eater braamfontein za flm rennie house eater braamfontein za flm rennie house eater braamfontein za flm rennie house eater braamfontein za seattle rennies house johannesburg za flm rennie house eater braamfontein za property rondebosch za seattle rennies house johannesburg za flm rennie house eater braamfontein za flm rennie house eater braamfontein za investecpb apple com bill itunes com ie bloom coffee studio cape town za seattle rennies house johannesburg za flm rennie house eater braamfontein za investecpb admed admed taxassist netcash seattle rennies house johannesburg za flm rennie house eater braamfontein za flm rennie house eater braamfontein za bolton rd collection parkview za virgin act rent levy mtn sp disc prem discovery vir atl vir atl gb amzn digital ch w kv us foundry cafe parktown nort za seattle rennies house johannesburg za flm rennie house ea

In [29]:
obj = TfidfVectorizer()
final_features = obj.fit_transform(new_features)
print(final_features[0])

  (0, 119)	0.0025916913455350794
  (0, 1)	0.0025916913455350794
  (0, 376)	0.005183382691070159
  (0, 138)	0.005183382691070159
  (0, 334)	0.0025916913455350794
  (0, 291)	0.0025916913455350794
  (0, 182)	0.005183382691070159
  (0, 201)	0.0025916913455350794
  (0, 280)	0.0025916913455350794
  (0, 265)	0.0025916913455350794
  (0, 88)	0.005183382691070159
  (0, 162)	0.0025916913455350794
  (0, 198)	0.0025916913455350794
  (0, 264)	0.0025916913455350794
  (0, 429)	0.0025916913455350794
  (0, 93)	0.0025916913455350794
  (0, 95)	0.005183382691070159
  (0, 50)	0.005183382691070159
  (0, 375)	0.007775074036605239
  (0, 145)	0.007775074036605239
  (0, 259)	0.010366765382140318
  (0, 17)	0.010366765382140318
  (0, 408)	0.005183382691070159
  (0, 189)	0.005183382691070159
  (0, 186)	0.005183382691070159
  :	:
  (0, 67)	0.015550148073210477
  (0, 26)	0.007775074036605239
  (0, 194)	0.0025916913455350794
  (0, 193)	0.005183382691070159
  (0, 341)	0.0025916913455350794
  (0, 312)	0.0025916913455350

In [30]:
X_train, X_test, y_train, y_test = train_test_split(final_features, y.values, test_size=0.20)

In [31]:
from sklearn.svm import LinearSVC
cls = LinearSVC(multi_class='crammer_singer', dual=False, fit_intercept=False)

In [32]:
y_pred = cls.fit(X_train, y_train.ravel())

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [33]:
pred1 = cls.predict(X_test)

In [34]:
print("Mean squared error: %.2f"
      % mean_squared_error(y_test, pred1))

Mean squared error: 10.29


In [35]:
print(classification_report(y_test, pred1))

              precision    recall  f1-score   support

           0       0.32      1.00      0.49        61
           1       0.00      0.00      0.00        47
           2       0.00      0.00      0.00        38
           3       0.00      0.00      0.00        10
           4       0.00      0.00      0.00         7
           5       0.00      0.00      0.00         7
           6       0.00      0.00      0.00         2
           7       0.00      0.00      0.00         5
           8       0.00      0.00      0.00         2
           9       0.00      0.00      0.00         4
          10       0.00      0.00      0.00         6

    accuracy                           0.32       189
   macro avg       0.03      0.09      0.04       189
weighted avg       0.10      0.32      0.16       189



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [36]:
y_predicted = pd.DataFrame({'PredictItem_ID':pred1})
y_predicted.head()

,PredictItem_ID
0,0
1,0
2,0
3,0
4,0


In [37]:
mapping = dict(y_unique[['New_ID','index']].values)
y_predicted['ItemCategory_New'] = y_predicted.PredictItem_ID.map(mapping)
y_predicted.head()

,PredictItem_ID,ItemCategory_New
0,0,transport
1,0,transport
2,0,transport
3,0,transport
4,0,transport


In [38]:

y_predicted = np.array(y_predicted.drop('PredictItem_ID', axis=1))
y_predicted[:5]

array([['transport'],
       ['transport'],
       ['transport'],
       ['transport'],
       ['transport']], dtype=object)

In [39]:
submission = pd.DataFrame({'description' : features2[X_train.shape[0]:],'category' : y_predicted.ravel()})

In [40]:
submission.head()

,description,category
0,FEE - PRE-PAID TOP UP,transport
1,CHECKERS BRAM 5326*9249 11 JAN,transport
2,C*ENGEN CONRA 5326*9249 13 JAN,transport
3,C*ENGEN PAARD 5326*9249 13 JAN,transport
4,I*MR D FOOD Z 5326*9249 14 JAN,transport
